In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
batch_size = 5000

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
  
        self.conv1 = nn.Conv2d(1, 32, 3, padding=0) 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=0)        
        self.pool1 = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout2d(p=0.3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 12 * 12, 1024)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.drop3 = nn.Dropout2d(p=0.5)
        self.fc3 = nn.Linear(512, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm1d(1024)
        self.bn5 = nn.BatchNorm1d(512)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.bn2(x)
        x = self.pool1(x)
        x = x.view(-1, 64 * 12 * 12)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.bn4(x)
        x = self.drop2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.bn5(x)
        x = self.drop3(x)
        x = self.fc3(x)
        return x

class custom_dataset(torch.utils.data.dataset.Dataset):
    def __init__(self, data, labels, transforms=None):
        self.data = data
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index):   
        dat = self.data[index]
        if self.transforms is not None:
            dat = self.transforms(dat)
        return (dat,self.labels[index])
   
    def __len__(self):
        return len(self.data)
      
                   
def lines_to_np_array(lines):
    return np.array([[i for i in line.split()] for line in lines])
with open('PATCH.amat') as f:
    lines = f.readlines()
x_test = lines_to_np_array(lines).astype('float32')

y_test = x_test[:, 784:]
x_test = x_test[:, :784]
X_test_final = []

for i in range(0, len(x_test)):
    X_test_final.append(torch.Tensor(x_test[i].reshape((1, 28, 28))))
    
transform_test = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])
test = custom_dataset(X_test_final,torch.LongTensor(y_test.reshape(len(y_test))), transforms=transform_test)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=0)
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ConvNet().to(device)
model.load_state_dict(torch.load(r"finalmodel.pt"))

model.eval()
with torch.no_grad():
    for data in test_loader:
        images, label = data
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        print("Predicted:", predicted, "\n")
       

Predicted: tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0') 

Predicted: tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0') 

Predicted: tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0') 

Predicted: tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0') 

Predicted: tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0') 

Predicted: tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0') 

Predicted: tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0') 

Predicted: tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0') 

Predicted: tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0') 

Predicted: tensor([0, 0, 0,  ..., 0, 0, 0], device='cuda:0') 

